In [1]:
import src

In [1]:
from src.algo.boost import GeneralBooster

inputs = GeneralBooster.df_input()
inputs

{'train':                      petro  coal  nonferrous  utility  public  steel  \
 datetime instrument                                                    
 20210924 1               0     0           0        0       0      0   
          2               0     0           0        0       0      0   
          4               0     0           0        0       0      0   
          6               0     0           0        0       0      0   
          8               0     0           0        0       0      0   
 ...                    ...   ...         ...      ...     ...    ...   
 20230418 688799          0     0           0        0       0      0   
          688800          0     0           0        0       0      0   
          688819          0     0           0        0       0      0   
          688981          0     0           0        0       0      0   
          689009          0     0           0        0       0      0   
 
                      chemical  construc

In [2]:
model1 = GeneralBooster('lgbm',**inputs).fit()
model2 = GeneralBooster('ada',**inputs).fit()

Round: 1, Est-Err: 0.4957, F_name: liquidity
Round: 2, Est-Err: 0.4992, F_name: size
Round: 3, Est-Err: 0.4996, F_name: a_industry
Round: 4, Est-Err: 0.4997, F_name: liquidity
Round: 5, Est-Err: 0.4999, F_name: a_momentum
Round: 6, Est-Err: 0.4999, F_name: beta
Round: 7, Est-Err: 0.4999, F_name: non_linear_size
Round: 8, Est-Err: 0.4999, F_name: a_industry
Round: 9, Est-Err: 0.4999, F_name: book_to_price
Round: 10, Est-Err: 0.4999, F_name: chemical
Round: 11, Est-Err: 0.4999, F_name: public
Round: 12, Est-Err: 0.4999, F_name: growth
Round: 13, Est-Err: 0.4999, F_name: liquidity
Round: 14, Est-Err: 0.4999, F_name: residual_volatility
Round: 15, Est-Err: 0.5000, F_name: momentum
Round: 16, Est-Err: 0.5000, F_name: leverage
Round: 17, Est-Err: 0.5000, F_name: a_industry
Round: 18, Est-Err: 0.4999, F_name: leisure
Round: 19, Est-Err: 0.5000, F_name: a_quality
Round: 20, Est-Err: 0.5000, F_name: media
Round: 21, Est-Err: 0.5000, F_name: telecom
Round: 22, Est-Err: 0.5000, F_name: residual_v

In [3]:
model2.fit()

Lgbm

In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_predict_data()
Predictor.update_factors()

Update Files
Sun Aug 18 20:40:48 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.01 Secs
Sun Aug 18 20:40:48 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Sun Aug 18 20:40:48 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Sun Aug 18 20:40:51 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.45 Secs
Sun Aug 18 20:40:55 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.67 Secs
Sun Aug 18 20:40:57 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240815.feather Done! Cost 0.12 Secs
Sun Aug 18 20:40:57 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240816.feather Done! Cost 0.04 Secs
Sun Aug 18 20:40:57 2024 : Updated ~ data\DataBase\DB_models\risk_cov\2024\risk_cov.20240816.feather Done! Cost 0.05 Secs
Sun Aug 18 20:40:57 2024 : Updated ~ data\DataBase\DB_models\risk_spec\2024\risk_spec.20240816.feath

KeyError: 'calendar'

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import DataAPI , Predictor
DataAPI.prepare_predict_data()
Predictor.update_factors()

predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 1.54 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 1.00 secs
 --> labels blocks merging (2)...... finished! Cost 0.20 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 3.87 secs
y blocks loading finished! Cost 7.08 secs
y blocks process...... finished! Cost 6.68 secs
y blocks masking...... finished! Cost 0.16 secs
y blocks saving ...... finished! Cost 0.19 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 14.26 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 1.49 secs
day blocks loading finished! Cost 1.53 secs
day blocks process...... finished! Cost 0.28 secs
day blocks masking...... finished! Cost 0.20

In [ ]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
